In [66]:
import LCMCore
reload("LCMCore")
module comms

import LCMCore: encode, decode, fingerprint

immutable CommsT
    utime::Int64
    format::String
    format_version_major::Int32
    format_version_minor::Int32
    data::Vector{UInt8}
end

fingerprint(::Type{CommsT}) = UInt8[211, 104, 224, 63, 51, 197, 104, 190]

function encode(io::IO, comms::CommsT)
    write(io, hton(comms.utime))
    write(io, hton(UInt32(length(comms.format) + 1)))
    write(io, comms.format)
    write(io, UInt8(0))
    write(io, hton(comms.format_version_major))
    write(io, hton(comms.format_version_minor))
    write(io, hton(Int32(length(comms.data))))
    write(io, hton.(comms.data))
end

function decode(io::IO, ::Type{CommsT})
    utime = Int64(ntoh(read(io, Int64)))
    formatlen = UInt32(ntoh(read(io, UInt32)))
    format = String(read(io, formatlen - 1))
    read(io, 1) # strip off null
    ver_maj = Int32(ntoh(read(io, Int32)))
    ver_min = Int32(ntoh(read(io, Int32)))
    num_bytes = Int32(ntoh(read(io, Int32)))
    data = String(read(io, UInt8, num_bytes))
    CommsT(utime, format, ver_maj, ver_min, data)
end

end

comms

In [67]:
lcm = LCMCore.LCM()

LCMCore.LCM(Ptr{Void} @0x00007fddb8346ac0,RawFD(113))

In [68]:
function onmsg(channel, msg)
    @show channel msg
end

onmsg (generic function with 1 method)

In [69]:
LCMCore.subscribe(lcm, "HELLO", onmsg, comms.CommsT)

Ptr{Void} @0x00007fddb8777b80

In [70]:
@async LCMCore.handle(lcm)

Task (runnable) @0x000000010d9221d0

In [72]:
LCMCore.publish(lcm, "HELLO", comms.CommsT(1234567, "format0", 11, 23, "Hello world"))

0